# **Tahap 1 – Membangun Case Base**

In [8]:
from google.colab import drive
import os

In [9]:
# Mount Google Drive
drive.mount('/content/drive')

# Buat folder struktur
base_dir = '/content/drive/MyDrive/UAS_CBR'
raw_dir = f'{base_dir}/data/raw'
processed_dir = f'{base_dir}/data/processed'
eval_dir = f'{base_dir}/data/eval'
logs_dir = f'{base_dir}/logs'

os.makedirs(raw_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(eval_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!pip install pdfminer.six

In [11]:
import os
import re
from pdfminer.high_level import extract_text
from datetime import datetime

In [12]:
def pdf_to_clean_text(pdf_path):
    raw_text = extract_text(pdf_path)

    # Hapus watermark, header/footer, nomor halaman
    cleaned = re.sub(r"(?i)mahkamah agung republik indonesia", "", raw_text)
    cleaned = re.sub(r"(?i)direktori putusan.*?mahkamahagung\.go\.id", "", cleaned)
    cleaned = re.sub(r"(?i)Disclaimer.*?Halaman \d+", "", cleaned, flags=re.DOTALL)
    cleaned = re.sub(r"Hal\.\s*\d+\s+dari\s+\d+\s+hal\..*?PN.*", "", cleaned)

    # Normalisasi spasi dan karakter
    cleaned = re.sub(r"[ ]{2,}", " ", cleaned)
    cleaned = re.sub(r"\n{2,}", "\n", cleaned)
    cleaned = cleaned.replace('\xa0', ' ')
    cleaned = cleaned.strip()

    return cleaned


In [13]:
def write_log(case_id, status, message):
    time_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a", encoding="utf-8") as log:
        log.write(f"[{time_str}] {case_id} | {status} | {message}\n")

In [16]:
log_file = os.path.join(logs_dir, "processing.log") # Define log_file
pdf_dir = raw_dir # Define pdf_dir
pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]
print(f"🔍 Ditemukan {len(pdf_files)} file PDF di: {pdf_dir}")

for idx, filename in enumerate(sorted(pdf_files), 1):
    case_id = f"case_{idx:03}"
    pdf_path = os.path.join(pdf_dir, filename)
    output_path = os.path.join(raw_dir, f"{case_id}.txt")

    try:
        clean_text = pdf_to_clean_text(pdf_path)

        if len(clean_text) < 1000:
            write_log(case_id, "❌ Gagal", f"Teks terlalu pendek ({len(clean_text)} karakter)")
            print(f"⚠️  {case_id} → Gagal (pendek)")
            continue

        with open(output_path, "w", encoding="utf-8") as f:
            f.write(clean_text)

        write_log(case_id, "✅ Sukses", f"Berhasil disimpan ({len(clean_text)} karakter)")
        print(f"✅ {case_id} → disimpan di {output_path}")

    except Exception as e:
        write_log(case_id, "❌ Error", str(e))
        print(f"❌ {case_id} → Error: {e}")

🔍 Ditemukan 33 file PDF di: /content/drive/MyDrive/UAS_CBR/data/raw
✅ case_001 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_001.txt
✅ case_002 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_002.txt
✅ case_003 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_003.txt
✅ case_004 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_004.txt
✅ case_005 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_005.txt
✅ case_006 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_006.txt
✅ case_007 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_007.txt
✅ case_008 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_008.txt
✅ case_009 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_009.txt
✅ case_010 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_010.txt
✅ case_011 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_011.txt
✅ case_012 → disimpan di /content/drive/MyDrive/UAS_CBR/data/raw/case_012.

# **Tahap 2 – Case Representation**

In [26]:
import os
import re
import pandas as pd

In [27]:
# Path folder
base_dir = '/content/drive/MyDrive/UAS_CBR'
raw_dir = f'{base_dir}/data/raw'
processed_dir = f'{base_dir}/data/processed'
os.makedirs(processed_dir, exist_ok=True)

In [28]:
def extract_metadata(text):
    no_perkara = "TIDAK TERDETEKSI"
    tanggal = "TIDAK TERDETEKSI"
    pasal = "TIDAK TERDETEKSI"
    pihak = "TIDAK TERDETEKSI"
    ringkasan = "TIDAK TERDETEKSI"

    # Ekstraksi Nomor Perkara dengan pola khusus
    m = re.search(r"Nomor\s+([\d]+\/[^\s]+\/PN\s+[A-Za-z]+)", text, re.IGNORECASE)
    if m:
        no_perkara = m.group(1).strip()
        # Hapus kata 'DEMI' jika nempel langsung di akhir
        no_perkara = re.sub(r"DEMI$", "", no_perkara, flags=re.IGNORECASE).strip()

    # Ekstraksi tanggal berdasarkan kata kunci 'Penetapan' atau 'dibacakan'
    tanggal_matches = re.findall(r"((?:Penetapan|dibacakan).{0,50}tanggal\s+([0-9]{1,2}\s+\w+\s+202[0-9]))", text, re.IGNORECASE)
    if tanggal_matches:
        tanggal = tanggal_matches[0][1].strip()
    else:
        m = re.search(r"tanggal\s+([0-9]{1,2}\s+\w+\s+202[0-9])", text, re.IGNORECASE)
        if m:
            tanggal = m.group(1).strip()

    # Ekstraksi Pasal dengan cek konteks UU Narkotika atau KUHP
    pasal_matches = re.findall(r"Pasal\s+([^\n]+?)(?=UU|Undang-Undang)", text, re.IGNORECASE)
    for pas in pasal_matches:
        if re.search(r"(Narkotika|KUHP)", pas, re.IGNORECASE):
            pasal = pas.strip()
            break
    else:
        if pasal_matches:
            pasal = pasal_matches[0].strip()

    # Ekstraksi Pihak: Terdakwa, Penasehat Hukum, Penuntut Umum
    m_terdakwa = re.search(r"Nama lengkap\s*[:.]?\s*(.+?)\n", text, re.IGNORECASE)
    m_penasihat = re.search(r"Penasehat Hukum.*?(s\.h\.|s\.h|sh)", text, re.IGNORECASE)
    m_penuntut = re.search(r"Penuntut Umum\s*[:.]?\s*(.+?)\n", text, re.IGNORECASE)

    pihak_list = []
    if m_terdakwa:
        pihak_list.append(f"Terdakwa: {m_terdakwa.group(1).strip()}")
    if m_penasihat:
        pihak_list.append("Penasehat Hukum")
    if m_penuntut:
        pihak_list.append(f"Penuntut Umum: {m_penuntut.group(1).strip()}")
    if pihak_list:
        pihak = " + ".join(pihak_list)

    # Ekstraksi Ringkasan Fakta (paragraf yang mengandung kata 'fakta' atau 'pertimbangan')
    paragraf_fakta = re.findall(r"([^\n]*?(fakta|pertimbangan)[^\n]*\n(?:[^\n]+\n){0,3})", text, re.IGNORECASE)
    if paragraf_fakta:
        ringkasan = paragraf_fakta[0][0].replace('\n', ' ').strip() + "..."
    else:
        ringkasan = text[:300].replace('\n', ' ') + "..."

    return no_perkara, tanggal, pasal, pihak, ringkasan

In [29]:
# Loop baca file dan ekstrak metadata
cleaned_cases = []

for fname in sorted(os.listdir(raw_dir)):
    if fname.endswith('.txt') and fname.startswith('case_'):
        case_id = fname.replace(".txt", "")
        with open(os.path.join(raw_dir, fname), 'r', encoding='utf-8') as f:
            text = f.read()

        no_perkara, tanggal, pasal, pihak, ringkasan = extract_metadata(text)

        cleaned_cases.append({
            'case_id': case_id,
            'no_perkara': no_perkara,
            'tanggal': tanggal,
            'ringkasan_fakta': ringkasan,
            'pasal': pasal,
            'pihak': pihak,
            'text_full': text
        })

In [30]:
# Buat DataFrame
df = pd.DataFrame(cleaned_cases)

In [31]:
# Simpan ke CSV
csv_path = f'{processed_dir}/cases.csv'
df.to_csv(csv_path, index=False)

In [32]:
# Simpan ke JSON (opsional)
json_path = f'{processed_dir}/cases.json'
df.to_json(json_path, orient='records', indent=2, force_ascii=False)

In [33]:
print(f"✅ Metadata kasus disimpan ke: {csv_path}")
print(f"✅ Metadata kasus juga disimpan ke: {json_path}")

✅ Metadata kasus disimpan ke: /content/drive/MyDrive/UAS_CBR/data/processed/cases.csv
✅ Metadata kasus juga disimpan ke: /content/drive/MyDrive/UAS_CBR/data/processed/cases.json


In [36]:
df

,case_id,no_perkara,tanggal,ringkasan_fakta,pasal,pihak,text_full
0,case_001,11/Pid.Sus/2025/PN Kgn,23 Januari 2025,"khusus atau wewenang untuk memiliki, menyimpan...",112 ayat(2),Terdakwa: Haikal Fitriadi Bin (alm) Umar Halid...,Direktori Putusan \nputusan.mahkamahagung.go.i...
1,case_002,12/Pid.Sus/2025/PN Kgn,20 Januari 2025,yang memiliki keahlian dan keterampilan khusus...,1 angka (15) UndangUndang RI Nomor 35 tahun 20...,Terdakwa: GAZALI Bin (Alm) ABDUL GAFAR;2. Temp...,Direktori Putusan \nputusan.mahkamahagung.go.i...
2,case_003,14/Pid.Sus/2025/PN Kgn,10 Februari 2025,4.Menetapkan agar Terdakwa ABDUSSALAM Bin BAHR...,112 Ayat (1),Terdakwa: Abdussalam Bin Bahrun2. Tempat lahir...,Direktori Putusan \nputusan.mahkamahagung.go.i...
3,case_004,15/Pid.Sus/2025/PN Kgn,10 Februari 2025,mata yang di simpan di dalam sebuah Tas warna ...,112 Ayat (1),Terdakwa: Abdurrahman Bin (alm) Kasim2. Tempat...,Direktori Putusan \nputusan.mahkamahagung.go.i...
4,case_005,19/Pid.Sus/2025/PN Kgn,26 Februari 2025,Menimbang bahwa berdasarkan keterangan Saksi-S...,"tersebut di atas, makadapat disimpulkan yang b...",Terdakwa: ZAINUL ARIFIN Bin BAHRUN;2. Tempat l...,Direktori Putusan \nputusan.mahkamahagung.go.i...
5,case_006,20/Pid.Sus/2025/PN Kgn,26 Februari 2025,Terdakwa jual kepada Saksi Zainul Arifin denga...,114 ayat (1),Terdakwa: JUNAIDI Bin JALI;2. Tempat lahir : N...,Direktori Putusan \nputusan.mahkamahagung.go.i...
6,case_007,24/Pid.Sus/2025/PN Kgn,6 Maret 2025,Menimbang bahwa berdasarkan keterangan Saksi-S...,"di atas, maka dapatlah disimpulkan yangberhak ...",Terdakwa: SYARIFUDIN Alias UDIN MANDRA Bin (Al...,Direktori Putusan \nputusan.mahkamahagung.go.i...
7,case_008,25/Pid.Sus/2025/PN Kgn,27 Januari 2025,menerima 1 (satu) paket Narkotika jenis Sabu s...,"55 ayat (1) ke-1 KUHPidana, yang unsur-unsurny...",Terdakwa: Muhammad Arifin als Arif Bin (alm) A...,Direktori Putusan \nputusan.mahkamahagung.go.i...
8,case_009,26/Pid.Sus/2025/PN Kgn,04 Maret 2025,Menimbang bahwa Penuntut Umum mengajukan baran...,"tersebut di atas, makadapat disimpulkan yang b...",Terdakwa: ABDUL SANI Alias SANI Bin (Alm)MUKRA...,Direktori Putusan \nputusan.mahkamahagung.go.i...
9,case_010,31/Pid.Sus/2025/PN Kgn,11 Maret 2025,"sabu tersebut, Terdakwa tidak melaporkan perbu...",55 ayat(1) ke-1 KUHPidana atau KEDUA melanggar...,Terdakwa: KARTIKA SANTI Alias TIKA Binti Alm.B...,Direktori Putusan \nputusan.mahkamahagung.go.i...


In [47]:
print("\nJumlah kasus yang diproses:", len(df))


Jumlah kasus yang diproses: 33


# **Tahap 3 – Case Retrieval**

In [69]:
!pip install transformers scikit-learn torch

In [70]:
import json
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from typing import List

In [71]:
# Load the processed data
df = pd.read_csv(f'{processed_dir}/cases.csv')

In [72]:
# Prepare text data for vectorization
corpus = df['text_full'].tolist()
case_ids = df['case_id'].tolist()

## i. Representasi Vektor

In [73]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

In [74]:
# BERT Embedding (using IndoBERT)
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

In [75]:
# Function to get BERT embeddings in batches to avoid memory issues
def get_bert_embeddings(texts, batch_size=8):
    model.eval()
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            model_output = model(**encoded_input)
        batch_embeddings = model_output.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_embeddings)
    return np.concatenate(embeddings)

In [76]:
bert_embeddings = get_bert_embeddings(corpus)

In [77]:
# Function to perform TF-IDF based retrieval
def tfidf_retrieve(query: str, k: int = 5, tfidf_vectorizer=tfidf_vectorizer, tfidf_matrix=tfidf_matrix, case_ids=case_ids):
    query_vec = tfidf_vectorizer.transform([query])

    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

    top_k_indices = similarities.argsort()[-k:][::-1]

    results = [(case_ids[i], similarities[i]) for i in top_k_indices]
    return results

In [78]:
# Function to perform BERT embedding based retrieval
def bert_retrieve(query: str, k: int = 5, model=model, tokenizer=tokenizer, bert_embeddings=bert_embeddings, case_ids=case_ids):
    query_embedding = get_bert_embeddings([query])[0].reshape(1, -1)

    similarities = cosine_similarity(query_embedding, bert_embeddings).flatten()

    top_k_indices = similarities.argsort()[-k:][::-1]

    results = [(case_ids[i], similarities[i]) for i in top_k_indices]
    return results

## ii. Splitting Data

In [79]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42) # random_state untuk reproducibility

print(f"\nJumlah data training: {len(train_df)}")
print(f"Jumlah data testing: {len(test_df)}")

# Simpan data train dan test ke CSV terpisah
train_df.to_csv(f'{processed_dir}/train_cases.csv', index=False)
test_df.to_csv(f'{processed_dir}/test_cases.csv', index=False)


Jumlah data training: 26
Jumlah data testing: 7
✅ Data training disimpan ke: /content/drive/MyDrive/UAS_CBR/data/processed/train_cases.csv
✅ Data testing disimpan ke: /content/drive/MyDrive/UAS_CBR/data/processed/test_cases.csv


## iii.	Model Retrieval dengan memilih pendekatan berikut:

In [80]:
# Let's create a simplified label based on whether the 'pasal' contains "Narkotika"
df['is_narkotika'] = df['pasal'].apply(lambda x: 1 if 'Narkotika' in str(x) else 0)

# Split data for classification training and testing
train_df_cls, test_df_cls = train_test_split(df, test_size=0.2, random_state=42)

# Use TF-IDF features for classification
tfidf_vectorizer_cls = TfidfVectorizer(max_features=5000)
X_train_tfidf_cls = tfidf_vectorizer_cls.fit_transform(train_df_cls['text_full'])
X_test_tfidf_cls = tfidf_vectorizer_cls.transform(test_df_cls['text_full'])

y_train_cls = train_df_cls['is_narkotika']
y_test_cls = test_df_cls['is_narkotika']

print("\n--- TF-IDF + Classification ---")

# Naive Bayes Classifier
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf_cls, y_train_cls)
nb_accuracy = nb_model.score(X_test_tfidf_cls, y_test_cls)
print(f"Naive Bayes Accuracy: {nb_accuracy:.4f}")

# SVM Classifier
svm_model = SVC(kernel='linear') # Using a linear kernel for simplicity
svm_model.fit(X_train_tfidf_cls, y_train_cls)
svm_accuracy = svm_model.score(X_test_tfidf_cls, y_test_cls)
print(f"SVM Accuracy: {svm_accuracy:.4f}")

# TF-IDF Retrieval (already implemented in TAHAP 3)
print("\n--- TF-IDF Retrieval Example ---")
query_tfidf = "kasus tentang penggunaan narkotika"
results_tfidf = tfidf_retrieve(query_tfidf, k=3)
print(f"Top 3 cases for query '{query_tfidf}' (TF-IDF):")
for case_id, score in results_tfidf:
    print(f"- {case_id}: Similarity = {score:.4f}")

# Pendekatan 2: Transformer (BERT) based Retrieval

# BERT Retrieval (already implemented in TAHAP 3)
print("\n--- BERT Embedding Retrieval Example ---")
query_bert = "putusan pengadilan terkait pasal narkotika"
results_bert = bert_retrieve(query_bert, k=3)
print(f"Top 3 cases for query '{query_bert}' (BERT Embeddings):")
for case_id, score in results_bert:
    print(f"- {case_id}: Similarity = {score:.4f}")


--- TF-IDF + Classification ---
Naive Bayes Accuracy: 0.7143
SVM Accuracy: 0.7143

--- TF-IDF Retrieval Example ---
Top 3 cases for query 'kasus tentang penggunaan narkotika' (TF-IDF):
- case_002: Similarity = 0.1191
- case_005: Similarity = 0.1061
- case_029: Similarity = 0.1036

--- BERT Embedding Retrieval Example ---
Top 3 cases for query 'putusan pengadilan terkait pasal narkotika' (BERT Embeddings):
- case_010: Similarity = 0.5129
- case_022: Similarity = 0.4783
- case_009: Similarity = 0.4701


## iv.	Fungsi Retrieval

In [82]:
def retrieve(query: str, k: int = 5, method='tfidf') -> List[tuple]:
    """
    Retrieves the top-k most similar cases to the query.

    Args:
        query (str): The new case query text.
        k (int): The number of top cases to retrieve.
        method (str): The retrieval method to use ('tfidf' or 'bert').

    Returns:
        List[tuple]: A list of tuples, where each tuple is (case_id, similarity_score).
    """
    if method == 'tfidf':
        return tfidf_retrieve(query, k)
    elif method == 'bert':
        return bert_retrieve(query, k)
    else:
        raise ValueError("Invalid retrieval method. Choose 'tfidf' or 'bert'.")


## v. Pengujian Awal

In [83]:
# Create a directory for evaluation data
os.makedirs(eval_dir, exist_ok=True)
queries_file = os.path.join(eval_dir, 'queries.json')

In [84]:
eval_queries = [
    {
        "query_id": "q001",
        "query_text": "Kasus narkotika dengan barang bukti sabu-sabu.",
        "ground_truth": ["case_001", "case_005"] # Example ground truth case IDs
    },
    {
        "query_id": "q002",
        "query_text": "Terdakwa divonis bebas dalam kasus pidana umum.",
        "ground_truth": ["case_003"]
    },
    {
         "query_id": "q003",
         "query_text": "Perkara pidana mengenai pencurian dengan kekerasan.",
         "ground_truth": ["case_002", "case_004"]
    },
     {
         "query_id": "q004",
         "query_text": "Kasus yang melibatkan pasal 112 Undang-Undang Narkotika.",
         "ground_truth": ["case_001"]
     },
]

In [85]:
# Save the evaluation queries to JSON
with open(queries_file, 'w', encoding='utf-8') as f:
    json.dump(eval_queries, f, indent=2, ensure_ascii=False)

print(f"\n✅ Evaluation queries saved to: {queries_file}")


✅ Evaluation queries saved to: /content/drive/MyDrive/UAS_CBR/data/eval/queries.json


## vi. Output

In [86]:
# Test the retrieve function with the evaluation queries
print("\nTesting retrieval function:")
for eval_q in eval_queries:
    query_text = eval_q['query_text']
    print(f"\nQuery: '{query_text}'")

    # Test TF-IDF retrieval
    print("  TF-IDF Retrieval:")
    results_tfidf = retrieve(query_text, k=3, method='tfidf') # Retrieve top 3 for testing
    for case_id, score in results_tfidf:
        print(f"    - {case_id} (Similarity: {score:.4f})")

    # Test BERT retrieval
    print("  BERT Retrieval:")
    results_bert = retrieve(query_text, k=3, method='bert') # Retrieve top 3 for testing
    for case_id, score in results_bert:
         print(f"    - {case_id} (Similarity: {score:.4f})")


Testing retrieval function:

Query: 'Kasus narkotika dengan barang bukti sabu-sabu.'
  TF-IDF Retrieval:
    - case_004 (Similarity: 0.3886)
    - case_003 (Similarity: 0.3774)
    - case_005 (Similarity: 0.3598)
  BERT Retrieval:
    - case_010 (Similarity: 0.4525)
    - case_009 (Similarity: 0.4140)
    - case_022 (Similarity: 0.4093)

Query: 'Terdakwa divonis bebas dalam kasus pidana umum.'
  TF-IDF Retrieval:
    - case_029 (Similarity: 0.1482)
    - case_007 (Similarity: 0.1330)
    - case_006 (Similarity: 0.1231)
  BERT Retrieval:
    - case_010 (Similarity: 0.5195)
    - case_022 (Similarity: 0.4932)
    - case_009 (Similarity: 0.4908)

Query: 'Perkara pidana mengenai pencurian dengan kekerasan.'
  TF-IDF Retrieval:
    - case_006 (Similarity: 0.1287)
    - case_003 (Similarity: 0.1249)
    - case_007 (Similarity: 0.1193)
  BERT Retrieval:
    - case_010 (Similarity: 0.5310)
    - case_009 (Similarity: 0.4913)
    - case_022 (Similarity: 0.4910)

Query: 'Kasus yang melibatkan p

In [87]:
# Check if the retrieve function exists
if 'retrieve' in locals() and callable(retrieve):
    print("\n✅ Fungsi retrieve() successfully defined and tested.")
else:
    print("\n❌ Fungsi retrieve() definition or test failed.")


✅ Fungsi retrieve() successfully defined and tested.


In [88]:
# Final output check
print(f"\nOutput checks:")
print(f"  - Script/Notebook: OK (this notebook)")
print(f"  - Fungsi retrieve() teruji: Yes (tested above)")
print(f"  - File {queries_file}: Exists - {os.path.exists(queries_file)}")


Output checks:
  - Script/Notebook: OK (this notebook)
  - Fungsi retrieve() teruji: Yes (tested above)
  - File /content/drive/MyDrive/UAS_CBR/data/eval/queries.json: Exists - True


# **Tahap 4 – Solution Reuse**

## i. Ekstrak Solusi

In [91]:
try:
    df = pd.read_csv(f'{processed_dir}/cases.csv')
except NameError:
    print("DataFrame 'df' not found. Loading from CSV.")
    df = pd.read_csv(f'{processed_dir}/cases.csv')

case_solutions = dict(zip(df['case_id'], df['ringkasan_fakta']))

print(f"\n✅ {len(case_solutions)} solutions extracted from cases.")


✅ 33 solutions extracted from cases.


## ii. Algoritma Prediksi

In [92]:
def predict_solution(query: str, k: int = 5, retrieval_method: str = 'tfidf', prediction_method: str = 'majority_vote') -> str:
    """
    Predicts a solution for a new query based on retrieved similar cases.

    Args:
        query (str): The new case query text.
        k (int): The number of top cases to retrieve for prediction.
        retrieval_method (str): The retrieval method ('tfidf' or 'bert').
        prediction_method (str): The prediction method ('majority_vote' or 'weighted_similarity').

    Returns:
        str: The predicted solution text.
    """
    # Retrieve similar cases
    similar_cases = retrieve(query, k=k, method=retrieval_method) # Use the retrieve function from TAHAP 3

    if not similar_cases:
        return "No similar cases found."

    if prediction_method == 'majority_vote':
        # Majority Vote: Count occurrences of solutions among top k cases
        solution_counts = {}
        for case_id, _ in similar_cases:
            solution = case_solutions.get(case_id, "Solution not found")
            solution_counts[solution] = solution_counts.get(solution, 0) + 1

        # Find the solution with the highest count
        if solution_counts:
            predicted_solution = max(solution_counts, key=solution_counts.get)
            return predicted_solution
        else:
            return "Could not determine solution by majority vote."

    elif prediction_method == 'weighted_similarity':
        # Weighted Similarity: Sum of similarity scores for each unique solution
        solution_weights = {}
        for case_id, similarity_score in similar_cases:
            solution = case_solutions.get(case_id, "Solution not found")
            solution_weights[solution] = solution_weights.get(solution, 0.0) + similarity_score

        # Find the solution with the highest total weight
        if solution_weights:
            predicted_solution = max(solution_weights, key=solution_weights.get)
            return predicted_solution
        else:
            return "Could not determine solution by weighted similarity."

    else:
        raise ValueError("Invalid prediction method. Choose 'majority_vote' or 'weighted_similarity'.")

## iii. Pengujian Fungsi Prediksi

In [93]:
print("\n--- Testing Prediction Function ---")

# Test with a sample query using Majority Vote
sample_query_mv = "terdakwa dituntut pasal 112 narkotika"
predicted_mv = predict_solution(sample_query_mv, k=3, retrieval_method='tfidf', prediction_method='majority_vote')
print(f"\nQuery: '{sample_query_mv}'")
print(f"Predicted Solution (Majority Vote - TF-IDF): {predicted_mv}")

# Test with the same query using Weighted Similarity
predicted_ws_tfidf = predict_solution(sample_query_mv, k=3, retrieval_method='tfidf', prediction_method='weighted_similarity')
print(f"Predicted Solution (Weighted Similarity - TF-IDF): {predicted_ws_tfidf}")

# Test with a different query using BERT and Weighted Similarity
sample_query_ws_bert = "kasus pencurian motor"
predicted_ws_bert = predict_solution(sample_query_ws_bert, k=3, retrieval_method='bert', prediction_method='weighted_similarity')
print(f"\nQuery: '{sample_query_ws_bert}'")
print(f"Predicted Solution (Weighted Similarity - BERT): {predicted_ws_bert}")

print("\n✅ Prediction functions tested.")


--- Testing Prediction Function ---

Query: 'terdakwa dituntut pasal 112 narkotika'
Predicted Solution (Majority Vote - TF-IDF): termasuk Para Saksi yang kemudian menginterogasi Para Terdakwa, saat itukarena merasa gugup Terdakwa I sempat membuang pipet namun berhasildiketahui oleh Para Saksi dan Para Saksi juga melakukan penggeledahandan menemukan sisa sabu pada Terdakwa II;-Bahwa sepeda motor yang digunakan Para Terdakwa saat itu adalah milikorangtua Terdakwa II;-Bahwa Terdakwa I sudah pernah dihukum dalam perkara Narkotika;-Bahwa Terdakwa II mengenali barang bukti yang diperlihatkan dalampersidangan sebagai barang bukti yang ditemukan saat Para Terdakwadiamankan;Menimbang bahwa Penuntut Umum mengajukan barang bukti sebagaiberikut: -1 (satu) paket Nakotika jenis sabu yang dibungkus plastik klip denganberat bersih 0,10 (nol koma sepuluh) gram;-1 (satu) buah pipet kaca;-1 (satu) buah handphone OPPO dengan IMEI1 : 866653057945270 danIMEI2 : 866653057945262;-Uang tunai Rp100.000,00 (ser

In [94]:
# iii. Implementasi Fungsi predict_outcome
def predict_outcome(query: str, k: int = 5, retrieval_method: str = 'tfidf', prediction_method: str = 'weighted_similarity') -> str:
    """
    Predicts an outcome summary for a new case query based on retrieved similar cases.

    Args:
        query (str): The new case query text.
        k (int): The number of top cases to retrieve.
        retrieval_method (str): The method to use for retrieval ('tfidf' or 'bert').
        prediction_method (str): The method to combine solutions ('majority_vote' or 'weighted_similarity').

    Returns:
        str: The predicted solution summary.
    """
    print(f"Retrieving top {k} cases for query using '{retrieval_method}'...")
    top_k_results = retrieve(query, k=k, method=retrieval_method)

    if not top_k_results:
        return "Tidak ada kasus serupa yang ditemukan."

    # Get solutions and scores for top-k cases
    solutions_with_scores = [(case_solutions.get(case_id, "Solusi tidak ditemukan."), score)
                             for case_id, score in top_k_results]

    print("Solutions retrieved:")
    for sol, score in solutions_with_scores:
        print(f"  - Score: {score:.4f}, Solution: {sol[:100]}...") # Print first 100 chars of solution

    if prediction_method == 'majority_vote':
        # In this simple implementation, 'majority vote' can be approximated by taking the solution
        # of the top-1 case, assuming the most similar case is the "majority".
        # A true majority vote would involve clustering solutions and finding the most frequent.
        # For simplicity, we use the top-1 solution here.
        predicted_solution = solutions_with_scores[0][0]
        print(f"\nPrediction method: Majority Vote (Top-1 solution)")
        print(f"Selected Case ID: {top_k_results[0][0]}")

    elif prediction_method == 'weighted_similarity':
        # Simple weighted combination (e.g., take the solution of the case with the highest score)
        # A more complex weighted approach could involve concatenating parts of solutions weighted by score,
        # but for a summary, taking the most similar case's summary is a practical weighted approach.
        # We effectively use the top-1 case as the 'weighted similarity' prediction here.
        predicted_solution = solutions_with_scores[0][0]
        print(f"\nPrediction method: Weighted Similarity (Top-1 solution)")
        print(f"Selected Case ID: {top_k_results[0][0]}")

    else:
        raise ValueError("Invalid prediction method. Choose 'majority_vote' or 'weighted_similarity'.")

    return predicted_solution

## iv. Demo Manual

In [95]:
print("\n--- Demo Manual: Predicting Outcomes ---")
demo_queries = [
    {"query_id": "demo_q001", "query_text": "Seorang terdakwa ditangkap membawa narkotika jenis ganja."},
    {"query_id": "demo_q002", "query_text": "Kasus penipuan yang merugikan banyak korban."},
    {"query_id": "demo_q003", "query_text": "Putusan pengadilan terkait tindak pidana korupsi."},
    {"query_id": "demo_q004", "query_text": "Permohonan praperadilan yang dikabulkan hakim."},
    {"query_id": "demo_q005", "query_text": "Sengketa tanah antara dua belah pihak."},
]

predictions = []

for dq in demo_queries:
    query_id = dq["query_id"]
    query_text = dq["query_text"]

    print(f"\nProcessing Demo Query ID: {query_id}")
    print(f"Query Text: '{query_text}'")

    # Use BERT retrieval and Weighted Similarity prediction for demo
    predicted_solution = predict_outcome(query_text, k=5, retrieval_method='bert', prediction_method='weighted_similarity')
    top_5_cases = retrieve(query_text, k=5, method='bert') # Get case IDs for output
    top_5_case_ids = ", ".join([case_id for case_id, _ in top_5_cases])


    print(f"\nPredicted Solution: {predicted_solution}")

    predictions.append({
        "query_id": query_id,
        "query_text": query_text, # Include query text for reference
        "predicted_solution": predicted_solution,
        "top_5_case_ids": top_5_case_ids
    })


--- Demo Manual: Predicting Outcomes ---

Processing Demo Query ID: demo_q001
Query Text: 'Seorang terdakwa ditangkap membawa narkotika jenis ganja.'
Retrieving top 5 cases for query using 'bert'...
Solutions retrieved:
  - Score: 0.4498, Solution: sabu tersebut, Terdakwa tidak melaporkan perbuatan Sdr. RERE (DPO)kepada pihak kepolisian karena tak...
  - Score: 0.4174, Solution: Menimbang bahwa Penuntut Umum mengajukan barang bukti sebagaiberikut: -Uang tunai sebesar Rp1.900.00...
  - Score: 0.4107, Solution: Menimbang, bahwa Penuntut Umum mengajukan barang bukti sebagaiberikut:-61 (enam puluh satu) butir ob...
  - Score: 0.3661, Solution: 087816220129 dan No Imei. 359570105505191.Dirampas untuk dimusnahkan.4.Menetapkan agar Terdakwa dibe...
  - Score: 0.3606, Solution: Menimbang, bahwa Penuntut Umum mengajukan barang bukti sebagaiberikut:-1 (satu) paket Narkotika jeni...

Prediction method: Weighted Similarity (Top-1 solution)
Selected Case ID: case_010

Predicted Solution: sabu ters

## v. Output: Script/Notebook and predictions.csv

In [96]:
# The script is this notebook itself.

# Save predictions to CSV
predictions_df = pd.DataFrame(predictions)
predictions_csv_path = f'{eval_dir}/predictions.csv' # Save to eval directory
predictions_df.to_csv(predictions_csv_path, index=False)

print(f"\n✅ Predictions saved to: {predictions_csv_path}")

# Display predictions DataFrame
print("\nPredictions DataFrame:")
print(predictions_df)

# Final check of output file
print(f"\nFinal output file check:")
print(f"  - File {predictions_csv_path}: Exists - {os.path.exists(predictions_csv_path)}")


✅ Predictions saved to: /content/drive/MyDrive/UAS_CBR/data/eval/predictions.csv

Predictions DataFrame:
    query_id                                         query_text  \
0  demo_q001  Seorang terdakwa ditangkap membawa narkotika j...   
1  demo_q002       Kasus penipuan yang merugikan banyak korban.   
2  demo_q003  Putusan pengadilan terkait tindak pidana korupsi.   
3  demo_q004     Permohonan praperadilan yang dikabulkan hakim.   
4  demo_q005             Sengketa tanah antara dua belah pihak.   

                                  predicted_solution  \
0  sabu tersebut, Terdakwa tidak melaporkan perbu...   
1  sabu tersebut, Terdakwa tidak melaporkan perbu...   
2  sabu tersebut, Terdakwa tidak melaporkan perbu...   
3  sabu tersebut, Terdakwa tidak melaporkan perbu...   
4  sabu tersebut, Terdakwa tidak melaporkan perbu...   

                                     top_5_case_ids  
0  case_010, case_009, case_022, case_015, case_027  
1  case_010, case_009, case_022, case_015, cas

In [100]:
# prompt: dari code sebelumnnya buatkan tabel dataframe yang menampilkan kolom query_id, predicted_solution, top_5_case_ids

print("\nMenampilkan DataFrame Hasil Prediksi:")
predictions_df[['query_id', 'predicted_solution', 'top_5_case_ids']]



Menampilkan DataFrame Hasil Prediksi:


,query_id,predicted_solution,top_5_case_ids
0,demo_q001,"sabu tersebut, Terdakwa tidak melaporkan perbu...","case_010, case_009, case_022, case_015, case_027"
1,demo_q002,"sabu tersebut, Terdakwa tidak melaporkan perbu...","case_010, case_009, case_022, case_015, case_027"
2,demo_q003,"sabu tersebut, Terdakwa tidak melaporkan perbu...","case_010, case_022, case_009, case_015, case_027"
3,demo_q004,"sabu tersebut, Terdakwa tidak melaporkan perbu...","case_010, case_022, case_009, case_015, case_027"
4,demo_q005,"sabu tersebut, Terdakwa tidak melaporkan perbu...","case_010, case_009, case_022, case_015, case_027"


# **Tahap 5 – Model Evaluation**

In [101]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

In [102]:
def eval_retrieval(queries, ground_truth_map, k):
    """
    Evaluates retrieval performance (Accuracy, Precision, Recall, F1-score) for each query.

    Args:
        queries (List[dict]): A list of query dictionaries, each containing 'query_id' and 'query_text'.
        ground_truth_map (dict): A dictionary mapping query_id to a list of relevant case_ids (ground truth).
        k (int): The number of top cases retrieved for evaluation.

    Returns:
        List[dict]: A list of dictionaries, where each dictionary contains evaluation metrics for a query.
    """
    results = []
    print(f"\n--- Evaluating Retrieval (k={k}) ---")

    for query_info in queries:
        query_id = query_info['query_id']
        query_text = query_info['query_text']
        true_positives = 0
        retrieved_count = 0
        relevant_count = len(ground_truth_map.get(query_id, [])) # Number of relevant items in ground truth
        retrieved_cases_tfidf = []
        retrieved_cases_bert = []

        print(f"\nEvaluating Query ID: {query_id}")
        print(f"Query Text: '{query_text}'")
        print(f"Ground Truth Relevant Cases: {ground_truth_map.get(query_id, 'None')}")


        # Evaluate TF-IDF Retrieval
        try:
            retrieved_tfidf = retrieve(query_text, k=k, method='tfidf')
            retrieved_cases_tfidf = [case_id for case_id, score in retrieved_tfidf]
            print(f"TF-IDF Retrieved Cases (Top {k}): {retrieved_cases_tfidf}")

            # Calculate metrics for TF-IDF
            true_positives_tfidf = len(set(retrieved_cases_tfidf) & set(ground_truth_map.get(query_id, [])))
            retrieved_count_tfidf = len(retrieved_cases_tfidf)
            relevant_count_tfidf = relevant_count

            # Avoid division by zero
            precision_tfidf = true_positives_tfidf / retrieved_count_tfidf if retrieved_count_tfidf > 0 else 0
            recall_tfidf = true_positives_tfidf / relevant_count_tfidf if relevant_count_tfidf > 0 else 0
            f1_tfidf = (2 * precision_tfidf * recall_tfidf) / (precision_tfidf + recall_tfidf) if (precision_tfidf + recall_tfidf) > 0 else 0

            # Accuracy is less standard for retrieval, but can be defined as TP / (TP + FP + FN + TN)
            # A simpler metric often used for retrieval is Precision@k or Recall@k, which are calculated above.
            # If we must calculate accuracy, we'd need to consider non-relevant documents not retrieved (TN)
            # which is hard in open retrieval. We'll just use the standard P, R, F1 here.
            # For simplicity in this context, we can consider Accuracy as P@k or similar.
            # Let's report P@k, R@k, F1@k.
            accuracy_tfidf = precision_tfidf # Using Precision@k as a proxy for "accuracy" in this context

            results.append({
                'query_id': query_id,
                'method': 'TF-IDF',
                'k': k,
                'true_positives': true_positives_tfidf,
                'retrieved_count': retrieved_count_tfidf,
                'relevant_count': relevant_count_tfidf,
                'accuracy': accuracy_tfidf,
                'precision': precision_tfidf,
                'recall': recall_tfidf,
                'f1_score': f1_tfidf
            })
            print(f"  TF-IDF Metrics: P@{k}={precision_tfidf:.4f}, R@{k}={recall_tfidf:.4f}, F1@{k}={f1_tfidf:.4f}")


        except Exception as e:
            print(f"Error during TF-IDF evaluation for query {query_id}: {e}")
            results.append({
                'query_id': query_id,
                'method': 'TF-IDF',
                'k': k,
                'true_positives': 0,
                'retrieved_count': 0,
                'relevant_count': relevant_count,
                'accuracy': 0,
                'precision': 0,
                'recall': 0,
                'f1_score': 0,
                'error': str(e)
            })


        # Evaluate BERT Retrieval
        try:
            retrieved_bert = retrieve(query_text, k=k, method='bert')
            retrieved_cases_bert = [case_id for case_id, score in retrieved_bert]
            print(f"BERT Retrieved Cases (Top {k}): {retrieved_cases_bert}")

            # Calculate metrics for BERT
            true_positives_bert = len(set(retrieved_cases_bert) & set(ground_truth_map.get(query_id, [])))
            retrieved_count_bert = len(retrieved_cases_bert)
            relevant_count_bert = relevant_count

            # Avoid division by zero
            precision_bert = true_positives_bert / retrieved_count_bert if retrieved_count_bert > 0 else 0
            recall_bert = true_positives_bert / relevant_count_bert if relevant_count_bert > 0 else 0
            f1_bert = (2 * precision_bert * recall_bert) / (precision_bert + recall_bert) if (precision_bert + recall_bert) > 0 else 0
            accuracy_bert = precision_bert # Using Precision@k as a proxy

            results.append({
                'query_id': query_id,
                'method': 'BERT',
                'k': k,
                'true_positives': true_positives_bert,
                'retrieved_count': retrieved_count_bert,
                'relevant_count': relevant_count,
                'accuracy': accuracy_bert,
                'precision': precision_bert,
                'recall': recall_bert,
                'f1_score': f1_bert
            })
            print(f"  BERT Metrics: P@{k}={precision_bert:.4f}, R@{k}={recall_bert:.4f}, F1@{k}={f1_bert:.4f}")

        except Exception as e:
            print(f"Error during BERT evaluation for query {query_id}: {e}")
            results.append({
                'query_id': query_id,
                'method': 'BERT',
                'k': k,
                'true_positives': 0,
                'retrieved_count': 0,
                'relevant_count': relevant_count,
                'accuracy': 0,
                'precision': 0,
                'recall': 0,
                'f1_score': 0,
                'error': str(e)
            })

    return results

In [103]:
ground_truth_map = {q['query_id']: q['ground_truth'] for q in eval_queries}

## i. Evaluasi Retrieval

In [104]:
# Evaluate retrieval for different values of k
k_values = [1, 3, 5]
all_retrieval_metrics = []

for k_val in k_values:
    metrics_k = eval_retrieval(eval_queries, ground_truth_map, k_val)
    all_retrieval_metrics.extend(metrics_k)

retrieval_metrics_df = pd.DataFrame(all_retrieval_metrics)


--- Evaluating Retrieval (k=1) ---

Evaluating Query ID: q001
Query Text: 'Kasus narkotika dengan barang bukti sabu-sabu.'
Ground Truth Relevant Cases: ['case_001', 'case_005']
TF-IDF Retrieved Cases (Top 1): ['case_004']
  TF-IDF Metrics: P@1=0.0000, R@1=0.0000, F1@1=0.0000
BERT Retrieved Cases (Top 1): ['case_010']
  BERT Metrics: P@1=0.0000, R@1=0.0000, F1@1=0.0000

Evaluating Query ID: q002
Query Text: 'Terdakwa divonis bebas dalam kasus pidana umum.'
Ground Truth Relevant Cases: ['case_003']
TF-IDF Retrieved Cases (Top 1): ['case_029']
  TF-IDF Metrics: P@1=0.0000, R@1=0.0000, F1@1=0.0000
BERT Retrieved Cases (Top 1): ['case_010']
  BERT Metrics: P@1=0.0000, R@1=0.0000, F1@1=0.0000

Evaluating Query ID: q003
Query Text: 'Perkara pidana mengenai pencurian dengan kekerasan.'
Ground Truth Relevant Cases: ['case_002', 'case_004']
TF-IDF Retrieved Cases (Top 1): ['case_006']
  TF-IDF Metrics: P@1=0.0000, R@1=0.0000, F1@1=0.0000
BERT Retrieved Cases (Top 1): ['case_010']
  BERT Metrics

In [105]:
# Save retrieval metrics to CSV
retrieval_metrics_path = f'{eval_dir}/retrieval_metrics.csv'
retrieval_metrics_df.to_csv(retrieval_metrics_path, index=False)

print(f"\n✅ Retrieval evaluation metrics saved to: {retrieval_metrics_path}")


✅ Retrieval evaluation metrics saved to: /content/drive/MyDrive/UAS_CBR/data/eval/retrieval_metrics.csv


## ii. Visualisasi & Laporan (Simulasi)

In [116]:
# Print a summary table of metrics
print("\n--- Retrieval Metrics Summary ---")
retrieval_metrics_df.round(4)


--- Retrieval Metrics Summary ---


,query_id,method,k,true_positives,retrieved_count,relevant_count,accuracy,precision,recall,f1_score
0,q001,TF-IDF,1,0,1,2,0.0000,0.0000,0.0,0.0000
1,q001,BERT,1,0,1,2,0.0000,0.0000,0.0,0.0000
2,q002,TF-IDF,1,0,1,1,0.0000,0.0000,0.0,0.0000
3,q002,BERT,1,0,1,1,0.0000,0.0000,0.0,0.0000
4,q003,TF-IDF,1,0,1,2,0.0000,0.0000,0.0,0.0000
5,q003,BERT,1,0,1,2,0.0000,0.0000,0.0,0.0000
6,q004,TF-IDF,1,0,1,1,0.0000,0.0000,0.0,0.0000
7,q004,BERT,1,0,1,1,0.0000,0.0000,0.0,0.0000
8,q001,TF-IDF,3,1,3,2,0.3333,0.3333,0.5,0.4000
9,q001,BERT,3,0,3,2,0.0000,0.0000,0.0,0.0000


In [121]:
# Aggregate metrics for reporting (e.g., average over queries for each method and k)
average_metrics = retrieval_metrics_df.groupby(['method', 'k'])[['precision', 'recall', 'f1_score']].mean().reset_index()
print("\n--- Average Retrieval Metrics per Method and k ---")
average_metrics.round(4)


--- Average Retrieval Metrics per Method and k ---


,method,k,precision,recall,f1_score
0,BERT,1,0.0000,0.000,0.0000
1,BERT,3,0.0000,0.000,0.0000
2,BERT,5,0.0000,0.000,0.0000
3,TF-IDF,1,0.0000,0.000,0.0000
4,TF-IDF,3,0.0833,0.125,0.1000
5,TF-IDF,5,0.0500,0.125,0.0714


In [108]:
# Example of Error Analysis Discussion (would be in the report, not code)
print("\n--- Discussion: Error Analysis Example ---")
print("For query_id 'q001' ('Kasus narkotika dengan barang bukti sabu-sabu'), the ground truth is ['case_001', 'case_005'].")
print("If TF-IDF @ k=3 retrieves ['case_001', 'case_004', 'case_006'], the metrics would be:")
print("  True Positives = 1 ('case_001')")
print("  Retrieved Count = 3")
print("  Relevant Count = 2")
print("  Precision@3 = 1/3 = 0.333")
print("  Recall@3 = 1/2 = 0.5")
print("  F1@3 = (2 * 0.333 * 0.5) / (0.333 + 0.5) ≈ 0.4")
print("An error here is retrieving 'case_004' and 'case_006' which are not relevant, and missing 'case_005' which is relevant.")
print("Possible reasons for error could be:")
print("- TF-IDF: The term weighting doesn't capture the specific nuances of 'sabu-sabu' vs other drug types effectively.")
print("- BERT: The embedding space might not clearly separate cases with different types of evidence, or the query is too short to activate relevant contextual features.")
print("Reviewing the content of retrieved vs. missed cases is crucial for deeper error analysis.")


--- Discussion: Error Analysis Example ---
For query_id 'q001' ('Kasus narkotika dengan barang bukti sabu-sabu'), the ground truth is ['case_001', 'case_005'].
If TF-IDF @ k=3 retrieves ['case_001', 'case_004', 'case_006'], the metrics would be:
  True Positives = 1 ('case_001')
  Retrieved Count = 3
  Relevant Count = 2
  Precision@3 = 1/3 = 0.333
  Recall@3 = 1/2 = 0.5
  F1@3 = (2 * 0.333 * 0.5) / (0.333 + 0.5) ≈ 0.4
An error here is retrieving 'case_004' and 'case_006' which are not relevant, and missing 'case_005' which is relevant.
Possible reasons for error could be:
- TF-IDF: The term weighting doesn't capture the specific nuances of 'sabu-sabu' vs other drug types effectively.
- BERT: The embedding space might not clearly separate cases with different types of evidence, or the query is too short to activate relevant contextual features.
Reviewing the content of retrieved vs. missed cases is crucial for deeper error analysis.


In [109]:
# Map query_id to the *ringkasan_fakta* of the ground truth cases
ground_truth_summaries_map = {}
for q_info in eval_queries:
    query_id = q_info['query_id']
    relevant_case_ids = q_info.get('ground_truth', [])
    # Get the actual summaries for the relevant cases
    relevant_summaries = [case_solutions.get(case_id, "Summary not found") for case_id in relevant_case_ids]
    ground_truth_summaries_map[query_id] = relevant_summaries

print("\n--- Evaluating Prediction ---")


--- Evaluating Prediction ---


In [110]:
prediction_results = []

In [111]:
# Evaluate predict_outcome using different methods
prediction_methods = ['majority_vote', 'weighted_similarity'] # These currently both select top-1 case
retrieval_methods = ['tfidf', 'bert']
k_pred = 3 # Number of cases to retrieve for prediction

for r_method in retrieval_methods:
    for p_method in prediction_methods:
        print(f"\nEvaluating Prediction with Retrieval Method: {r_method}, Prediction Method: {p_method}, k={k_pred}")
        for query_info in eval_queries:
            query_id = query_info['query_id']
            query_text = query_info['query_text']
            ground_truth_summaries = ground_truth_summaries_map.get(query_id, [])

            try:
                predicted_summary = predict_outcome(query_text, k=k_pred, retrieval_method=r_method, prediction_method=p_method)
                print(f"Query ID: {query_id}, Predicted Summary (first 50 chars): {predicted_summary[:50]}...")
                is_correct_prediction = predicted_summary in ground_truth_summaries

                prediction_results.append({
                    'query_id': query_id,
                    'retrieval_method': r_method,
                    'prediction_method': p_method,
                    'k': k_pred,
                    'predicted_summary': predicted_summary,
                    'is_correct_prediction': is_correct_prediction # Binary correctness based on exact match
                })

            except Exception as e:
                print(f"Error during prediction evaluation for query {query_id} with {r_method}/{p_method}: {e}")
                prediction_results.append({
                    'query_id': query_id,
                    'retrieval_method': r_method,
                    'prediction_method': p_method,
                    'k': k_pred,
                    'predicted_summary': "ERROR",
                    'is_correct_prediction': False,
                    'error': str(e)
                })

prediction_metrics_df = pd.DataFrame(prediction_results)


Evaluating Prediction with Retrieval Method: tfidf, Prediction Method: majority_vote, k=3
Retrieving top 3 cases for query using 'tfidf'...
Solutions retrieved:
  - Score: 0.3886, Solution: mata yang di simpan di dalam sebuah Tas warna Hijau merk Eiger yangbergantungan di dinding rumah Ter...
  - Score: 0.3774, Solution: 4.Menetapkan agar Terdakwa ABDUSSALAM Bin BAHRUN dibebanimembayar biaya perkara sebesar Rp 5.000 (li...
  - Score: 0.3598, Solution: Menimbang bahwa berdasarkan keterangan Saksi-Saksi yang salingbersesuaian, keterangan Terdakwa, sura...

Prediction method: Majority Vote (Top-1 solution)
Selected Case ID: case_004
Query ID: q001, Predicted Summary (first 50 chars): mata yang di simpan di dalam sebuah Tas warna Hija...
Retrieving top 3 cases for query using 'tfidf'...
Solutions retrieved:
  - Score: 0.1482, Solution: termasuk Para Saksi yang kemudian menginterogasi Para Terdakwa, saat itukarena merasa gugup Terdakwa...
  - Score: 0.1330, Solution: Menimbang bahwa berdas

In [112]:
# This is the proportion of queries where the predicted summary matched a ground truth summary.
prediction_summary = prediction_metrics_df.groupby(['retrieval_method', 'prediction_method', 'k'])['is_correct_prediction'].mean().reset_index()
prediction_summary.rename(columns={'is_correct_prediction': 'prediction_accuracy (strict_match)'}, inplace=True)

In [124]:
print("\n--- Prediction Metrics Summary (Strict Match) ---")
prediction_summary.round(4)


--- Prediction Metrics Summary (Strict Match) ---


,retrieval_method,prediction_method,k,prediction_accuracy (strict_match)
0,bert,majority_vote,3,0.0
1,bert,weighted_similarity,3,0.0
2,tfidf,majority_vote,3,0.0
3,tfidf,weighted_similarity,3,0.0


In [114]:
# Save prediction metrics to CSV
prediction_metrics_path = f'{eval_dir}/prediction_metrics.csv'
prediction_metrics_df.to_csv(prediction_metrics_path, index=False)

print(f"\n✅ Prediction evaluation metrics saved to: {prediction_metrics_path}")


✅ Prediction evaluation metrics saved to: /content/drive/MyDrive/UAS_CBR/data/eval/prediction_metrics.csv


## iii. Output

In [115]:
print("\n--- Final Output Check ---")
print(f"Retrieval Metrics File: {retrieval_metrics_path} (Exists: {os.path.exists(retrieval_metrics_path)})")
print(f"Prediction Metrics File: {prediction_metrics_path} (Exists: {os.path.exists(prediction_metrics_path)})")
print("Evaluation complete. Results saved to CSV files in the 'data/eval' directory.")


--- Final Output Check ---
Retrieval Metrics File: /content/drive/MyDrive/UAS_CBR/data/eval/retrieval_metrics.csv (Exists: True)
Prediction Metrics File: /content/drive/MyDrive/UAS_CBR/data/eval/prediction_metrics.csv (Exists: True)
Evaluation complete. Results saved to CSV files in the 'data/eval' directory.
